In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os 
import json


kaggle_json = './drive/MyDrive/kaggle.json'
with open(kaggle_json, 'r') as f:
    json_data = json.load(f)
    os.environ['KAGGLE_USERNAME'] = json_data['username']
    os.environ['KAGGLE_KEY'] = json_data['key']


In [ ]:
import os

# 폴더 생성 (기존재시 pass)
os.makedirs('./input',  exist_ok=True)
os.makedirs('./output', exist_ok=True)

In [ ]:
# 데이터 다운로드(api key 세팅 참조 링크 : https://github.com/Kaggle/kaggle-api)
!kaggle competitions download -c plant-pathology-2020-fgvc7

plant-pathology-2020-fgvc7.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# 압축 해제 후 input 에 train.csv test.csv  넣기
import zipfile
zipfile.ZipFile(f'plant-pathology-2020-fgvc7.zip').extractall('./input')

In [ ]:
import torch
import random
import numpy as np
import os

# 시드값 고정
seed = 50
os.environ['PYTHONSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

In [ ]:
import pandas as pd

# 데이터 경로
train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')
submission = pd.read_csv('./input/sample_submission.csv')

In [ ]:
from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 분리
train, valid = train_test_split(
    train, test_size=0.1,
    stratify=train[['healthy', 'multiple_diseases', 'rust', 'scab']],
    random_state=50)

In [ ]:
import cv2
from torch.utils.data import Dataset # 데이터 생성을 위한 클래스
import numpy as np

class ImageDataset(Dataset):
    # 초기화 메서드 (생성자)
    def __init__(self, df, img_dir ='./', transform=None, is_test=False):
        super().__init__() # 상속받은 Dataset의 __init__() 메서드 호출
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
    
    # 데이터셋 크기 반환 메서드    
    def __len__(self):
        return len(self.df)
    
    # 인덱스(idx)에 해당하는 데이터 반환 메서드
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0] # 이미지 ID
        img_path = self.img_dir + img_id + '.jpg' # 이미지 파일 경로
        image = cv2.imread(img_path) # 이미지 파일 읽기
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # 이미지 색상 보정
        
        if self.transform is not None:
            image = self.transform(image=image)['image']
            
        # 테스트 데이터면 이미지 데이터만 반환, 그렇지 않으면 타깃값도 반환 
        if self.is_test:
            return image # 테스트 용일때
        else:
            # 타깃값 4개 중 가장 큰 값의 인덱스
            label = np.argmax(self.df.iloc[idx, 1:5]) 
            return image, label 
    

In [ ]:
# 이미지변환을 위한 모듈
import albumentations as A
from albumentations.pytorch import ToTensorV2

# 훈련 데이터용 변환기
transform_train = A.Compose([
    A.Resize(450, 650), # 이미지 크기 조절
    A.RandomBrightnessContrast(
        brightness_limit=0.2, # 밝기 대비 조절
        contrast_limit=0.2,
        p=0.3
    ),
    A.VerticalFlip(p=0.2), # 상하 대칭 변환
    A.HorizontalFlip(p=0.5), # 좌우 대칭 변환
    A.ShiftScaleRotate(
        shift_limit=0.1,
        scale_limit=0.2,
        rotate_limit=30, p=0.3
    ),
    A.OneOf([A.Emboss(p=1),
             A.Sharpen(p=1),
             A.Blur(p=1)], p=0.3),
    A.PiecewiseAffine(p=0.3), # 어파인 변환
    A.Normalize(),
    ToTensorV2() # 텐서로 변환
])

In [ ]:
# 검증 및 테스트 데이터용 변환기
transform_test = A.Compose([
    A.Resize(450, 650), # 크기 조절
    A.Normalize(), # 정규화
    ToTensorV2() # 텐서로 변환 
])

In [ ]:
img_dir = './input/images/'

dataset_train = ImageDataset(train, img_dir=img_dir, transform=transform_train)
dataset_valid = ImageDataset(valid, img_dir=img_dir, transform=transform_test)

In [ ]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 4

loader_train = DataLoader(dataset_train, batch_size=batch_size,
                          shuffle=True, worker_init_fn=seed_worker,
                          generator=g, num_workers=2)

loader_valid = DataLoader(dataset_valid, batch_size=batch_size,
                          shuffle=False, worker_init_fn=seed_worker,
                          generator=g, num_workers=2)

In [ ]:
!pip install efficientnet-pytorch==0.7.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
from efficientnet_pytorch import EfficientNet

# 사전 훈련된 efficientnet-b7 모델 불러오기
model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=4)
model = model.to(device) # 장비할당 

Loaded pretrained weights for efficientnet-b7


**efficientnet-b7 출력값 개수를 설정하는 또 다른 방법**
```
import torch.nn as nn

# 사전 훈련된 efficientnet-b7 모델 불러오기
model = EfficientNet.from_pretrained('efficientnet-b7')

# 불러온 efficientnet-b7 모델의 마지막 계층 수정
model._fc = nn.Sequential(
    nn.Linear(model._fc.in_features, model._fc.out_features), # 2560 -> 1000
    nn.ReLU(), # 활성화 함수 
    nn.Dropout(p=0.5), # 50% 드롭아웃
    nn.Linear(model._fc.out_features, 4) # 1000 -> 4 
)
```

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**손실 함수와 옵티마이저 설정**

In [ ]:
import torch.nn as nn

# 손실 함수
criterion = nn.CrossEntropyLoss()
# 옵티마이저
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0006, weight_decay=0.0001)

In [ ]:
from transformers import get_cosine_schedule_with_warmup
epochs = 39 # 총 에폭

# 스케줄러 생성
scheduler = get_cosine_schedule_with_warmup(optimizer,
                                   num_warmup_steps=len(loader_train)*3,
                                   num_training_steps=len(loader_train)*epochs)

In [ ]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계싼 함수
from tqdm.notebook import tqdm # 진행률 표시 막대

# 총 에폭만큼 반복
for epoch in range(epochs):
    # == [ 훈련 ] =================================
    model.train() # 모델을 훈련 상태로 설정
    epoch_train_loss = 0  # 에폭별 손실값 초기화 (훈련 데이터용)
    
    # '반복 횟수' 만큼 반복
    for images, labels in tqdm(loader_train): # 
        # 이미지, 레이블(타깃값) 데이터 미니배치를 장비에 할당
        images = images.to(device)
        labels = labels.to(device)
        
        # 옵티마이저 내 기울기 초기화
        optimizer.zero_grad()
        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 손실 함수를 활용해 outputs 와 labels 의 손실값 계산
        loss = criterion(outputs, labels)
        # 현재 배치에서의 손실 추가 (훈련 데이터용 )
        epoch_train_loss += loss.item()
        loss.backward() # 역전파 수행
        optimizer.step() # 가중치 갱신
        scheduler.step()
    # 훈련 데이터 손실값 출력
    print(f'에폭 [{epoch+1}/{epochs}] - 훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')
    
    # == [ 검증 ] ====
    model.eval() # 모델을 평가 상태로 설정
    epoch_valid_loss = 0 # 에폭별 손실값 초기화(검증 데이터용)
    preds_list = [] # 예측 확률값 저장용 리스트 초기화 
    true_onehot_list = []
    
    with torch.no_grad(): # 기울기 계산 비활성화
        # 미니배치 단위로 검증
        for images, labels in loader_valid:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            epoch_valid_loss += loss.item()
            
            preds = torch.softmax(outputs.cpu(), dim=1).numpy() # 예측 확률값
            # 실젯값 (원-핫 인코딩 형식)
            true_onehot = torch.eye(4)[labels.to('cpu')].numpy()
            # 예측 확률값과 실젯값 저장
            preds_list.extend(preds)
            true_onehot_list.extend(true_onehot)
        # 검증 데이터 손실값 및 ROC AUC 점수 출력
        print(f'''에폭 [{epoch+1}/{epochs}]
              검증 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f} 
              검증 데이터 ROC AUC : {roc_auc_score(true_onehot_list, preds_list):.4f}''')

  0%|          | 0/410 [00:00<?, ?it/s]

에폭 [1/39] - 훈련 데이터 손실값 : 0.8211
에폭 [1/39]
              검증 데이터 손실값 : 0.3749 
              검증 데이터 ROC AUC : 0.9113


  0%|          | 0/410 [00:00<?, ?it/s]

에폭 [2/39] - 훈련 데이터 손실값 : 0.4252
에폭 [2/39]
              검증 데이터 손실값 : 0.6260 
              검증 데이터 ROC AUC : 0.9409


  0%|          | 0/410 [00:00<?, ?it/s]

...

  0%|          | 0/410 [00:00<?, ?it/s]

에폭 [20/39] - 훈련 데이터 손실값 : 0.0917
에폭 [20/39]
              검증 데이터 손실값 : 0.2466 
              검증 데이터 ROC AUC : 0.9609


  0%|          | 0/410 [00:00<?, ?it/s]

에폭 [21/39] - 훈련 데이터 손실값 : 0.0635
에폭 [21/39]
              검증 데이터 손실값 : 0.2539 
              검증 데이터 ROC AUC : 0.9691


  0%|          | 0/410 [00:00<?, ?it/s]

In [ ]:
torch.save(model.state_dict(), './model.pth')

In [ ]:
with torch.no_grad(): # 기울기 계산 비활성화
    # 미니배치 단위로 검증
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        epoch_valid_loss += loss.item()
        
        preds = torch.softmax(outputs.cpu(), dim=1).numpy() # 예측 확률값
        # 실젯값 (원-핫 인코딩 형식)
        true_onehot = torch.eye(4)[labels.to('cpu')].numpy()
        break
        # 예측 확률값과 실젯값 저장
        preds_list.extend(preds)
        true_onehot_list.extend(true_onehot)

In [ ]:
# 테스트 데이터 원본용 데이터셋 및 데이터 로데
dataset_test = ImageDataset(test, img_dir=img_dir,
                            transform=transform_test, is_test=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size,
                         shuffle=False, worker_init_fn=seed_worker,
                         generator=g, num_workers=2)

# TTA 용 데이터셋 및 데이터로더 
dataset_test = ImageDataset(test, img_dir=img_dir,
                            transform=transform_train, is_test=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size,
                         shuffle=False, worker_init_fn=seed_worker,
                         generator=g, num_workers=2)

In [ ]:
model.eval()

preds_test = np.zeros((len(test), 4))

with torch.no_grad():
    for i, images in enumerate(loader_test):
        images = images.to(device)
        outputs = model(images)
        # 타깃 예측 확률 
        preds_part = torch.softmax(outputs.cpu(), dim=1).squeeze().numpy() 
        preds_test[i*batch_size : (i+1)*batch_size] += preds_part

In [ ]:
submission_test = submission.copy()

submission_test[['healthy', 'multiple_diseases', 'rust', 'scab']] = preds_test

In [ ]:
num_TTA = 7 # TTA 횟수

preds_tta = np.zeros((len(test), 4))

with torch.no_grad():
    for i, images in enumerate(loader_test):
        images = images.to(device)
        outputs = model(images)
        # 타깃 예측 확률 
        preds_part = torch.softmax(outputs.cpu(), dim=1).squeeze().numpy() 
        preds_tta[i*batch_size : (i+1)*batch_size] += preds_part

In [ ]:
preds_tta /= num_TTA

In [ ]:
submission_tta = submission.copy()

submission_tta[['healthy', 'multiple_diseases', 'rust', 'scab']] = preds_tta

In [ ]:
submission_test.to_csv('./output/submission_test.csv',index=False)
submission_tta.to_csv('./output/submission_tta.csv',index=False)

In [ ]:
def apply_label_smoothing(df, target, alpha, threshold):
    # 타깃값 복사
    df_target = df[target].copy()
    k = len(target)
    
    for idx, row in df_target.iterrows():
        if (row > threshold).any():
            row = (1-alpha)*row + alpha/k
            df_target.iloc[idx] = row
    return df_target

In [ ]:
alpha = 0.001 # 레이블 스무딩 강도
threshold = 0.999 # 레이블 스무딩을 적용할 임계값 

# 레이블 스무딩을 적용하기 위해 Dataframe 복사
submission_test_ls = submission_test.copy()
submission_tta_ls = submission_tta.copy()

target = ['healthy', 'multiple_diseases', 'rust', 'scab'] # 타깃값 열 이름 

# 레이블 스무딩 적용
submission[target] = apply_label_smoothing(submission_test_ls, target, alpha, threshold)
submission_tta_ls[target] = apply_label_smoothing(submission_tta_ls, target, alpha, threshold)

submission_test_ls.to_csv('./output/submission_test_ls.csv', index=False)
submission_tta_ls.to_csv('./output/submission_tta_ls.csv', index=False)

In [ ]:
!kaggle competitions submit -c plant-pathology-2020-fgvc7 -f ./output/submission.csv -m submit_in_colab

100% 163k/163k [00:01<00:00, 116kB/s]
Successfully submitted to Plant Pathology 2020 - FGVC7